In [1]:
import json
import os
import pandas as pd

from dotenv import load_dotenv
from openrouteservice import client, places

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = []
for i in range(1,17): 
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
category_id = 280 # park
buffer_size = 2000   # between 1-2000
park_dict = {}

In [ ]:
## read apartment data
property_data = json.load(open( f"{output_dir}properties_processed.json" ))

In [ ]:
token = tokens[0]

## set x and y
x = len(park_dict)
y = len(park_dict)+100

for index in range(x,y):
    if index > len(property_data['Coordinates'].keys()):
        break
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [backwards[1], backwards[0]]

    ## query code
    ors = client.Client(key=token)
    query = {'request': 'pois',
            'geojson': {'type':'Point','coordinates':coords},
            'buffer': buffer_size,
            'filter_category_ids': category_id,
            'sortby':'distance',
            'limit': 4}
    features = ors.places(**query)['features']

    ## distances - direct meters
    distances = []
    for poi in features:
        distances.append(poi['properties']['distance'])
        
    ## add information
    park_dict[str(index)] = distances

In [ ]:
property_data['Park'] = park_dict

## write json
property_data['Park'] = park_dict
json.dump(property_data, open(f"{output_dir}num_park.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}num_park.json")
data.to_csv(f"{output_dir}num_park.csv")